In [1]:
import pandas as pd 
import scipy as sp
from scipy import stats
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.kernel_ridge import KernelRidge
import math
from scipy.signal import argrelextrema

In [2]:
# get conditional and unconditional return
# all return are log-return
def getCondRet(rets,windowIds,window):
    '''
    window = l+d
    windowIds = array of beginning time id of a time window containing a pattern
    '''
    return rets[windowIds+window+1]

def getUncondRet(rets):
    pass
    

    
# standardize return - for each stock
def standardize(rets):
    return (rets - np.mean(rets))/np.std(rets)
    
    




In [3]:
# goodness-of-fit tests
from scipy import stats
def gof_ks(rets1,rets2):
    '''
    two samples can be of different size
    '''
    ks,p = stats.ks_2samp(rets1, rets2)
    return ks,p

In [4]:
from scipy.stats import norm
from scipy.stats import chi2



def gof_q(rets1,rets2):
    '''
    rets1 - conditional
    rets2 - unconditional
    '''
    n = len(rets1)
    decile = np.array([10,20,30,40,50,60,70,80,90,100])
    dec_uncond = np.percentile(rets2,decile)
    dec_cond = []
    delta = np.zeros(10)

    r = rets1[(rets1<=dec_uncond[0])]
    dec_cond.append(r)
    delta[0] = len(r)

    for j in range(9):
        r = rets1[(rets1<=dec_uncond[j+1]) & (rets1>=dec_uncond[j])]
        dec_cond.append(r)
        delta[j+1] = len(r)

    # for each decile  
    delta_pct = delta/n
    z = (delta_pct - 0.1)*np.sqrt(n)/(0.1*0.9)
    pvalue_delta = (1-norm.cdf(abs(z)))*2

    # Q-stats and chi-square distribution
    q = np.sum((delta - 0.1*n)**2/(0.1*n))
    pvalue_q = 1-chi2.cdf(q, 9)
    
    return delta_pct,pvalue_delta,q,pvalue_q

In [5]:
def makeGofTable(rets1,rets2,idx):
    
    delta_pct,pvalue_delta,q,pvalue_q = gof_q(rets1,rets2)
    ks,p = gof_ks(rets1,rets2)
    dat = np.append(np.append(delta_pct,q),ks)
    pv = np.append(np.append(pvalue_delta,pvalue_q),p)
    col = [str(x) for x in np.arange(1,11)]
    col.append('Qstat')
    col.append('KSstat')
    arrays = [[idx,idx], ['value', 'p-value']]
    mindex = pd.MultiIndex.from_arrays(arrays, names=('pattern', 'stats'))
    table = pd.DataFrame([dat,pv],columns=col,index=mindex)
    table = table.round(3)
    return table



In [6]:
rets1 = np.random.normal(0,1,100)
rets2 = np.random.normal(1,1,300)
table = makeGofTable(rets1,rets2,'HS')
table

1    2      3      4      5     6      7     8     9  \
pattern stats                                                              
HS      value    0.43  0.1  0.130  0.090  0.090  0.03  0.070  0.04  0.01   
        p-value  0.00  1.0  0.001  0.267  0.267  0.00  0.001  0.00  0.00   

                   10  Qstat  KSstat  
pattern stats                         
HS      value    0.01  135.6   0.383  
        p-value  0.00    0.0   0.000

In [7]:
## old code

def makeGOFTable(delta_pct,pvalue_delta,q,pvalue_q,idx):
    dat = np.append(delta_pct,q)
    pv = np.append(pvalue_delta,pvalue_q)
    col = [str(x) for x in np.arange(1,11)]
    col.append('Qstat')
    arrays = [[idx,idx], ['value', 'p-value']]
    mindex = pd.MultiIndex.from_arrays(arrays, names=('pattern', 'stats'))
    table = pd.DataFrame([dat,pv],columns=col,index=mindex)
    table = table.round(3)
    return table

table = makeGOFTable(delta_pct,pvalue_delta,q,pvalue_q,'HS')
table

NameError: name 'delta_pct' is not defined